rule_mapping.csv는 playlist_seed = mood + genre + goal 식으로 단순 문자열로 굉장히 단순함

bpm, energy, vocal 변수들도 사용가능함

단순 seed 문자열 대신 **복합 검색 쿼리**(/search용 query string)로 업그레이드 할 거임

Spotify /search는 type=playlist나 type=track 지정에 따라 꽤 높은 정확도로 관련 콘텐츠를 찾아줌

potify의 /search 엔드포인트는 텍스트 기반 쿼리만 받는 구조

→ 즉, “bpm=85” 같은 수치는 직접 쿼리 불가지만,
이를 자연어 기반으로 치환해 검색 정확도를 높일 수 있음

In [7]:
import pandas as pd

# CSV 파일 경로 (쥬피터 노트북 파일과 같은 폴더에 있으면 그냥 이름만 써도 됨)
df = pd.read_csv("rule_mapping_v1.csv", encoding="utf-8-sig")

# 제대로 불러왔는지 확인
print("행 개수:", len(df))
print("열 목록:", df.columns.tolist())


행 개수: 210
열 목록: ['location', 'db_band', 'goal', 'bpm_min', 'bpm_max', 'energy_min', 'energy_max', 'mood', 'genre_primary', 'genre_secondary', 'vocal', 'playlist_seed', 'notes']


**rule_mapping__v2.csv** 생성

개선 (playlist_seed → search_query 업그레이드) -> 고도화해서 추천정확도 높임

검색어를 그대로 스포티파이 주소 뒤에 붙인 상태

주소 누르면 실제 Spotify UI에서는 기본적으로 playlist + track + artist + album을 한꺼번에 보여주는 통합 검색(all) 모드로 넘어감

build_search_query.py: rule_mapping_v1.csv → rule_mapping_v2.csv 변환하는 기본 스크립트

In [14]:

def make_search_query(row):
    parts = []
    
    # 1) 기본 구성요소
    parts.append(row["genre_primary"])
    if pd.notna(row["genre_secondary"]):
        parts.append(row["genre_secondary"])
    parts.append(row["goal"])
    parts.append(row["mood"])

    # 2) Vocal 필터링
    if "instrumental" in row["vocal"]:
        parts.append("instrumental")
        parts.append("-live")
        parts.append("-remix")
    elif "no" in row["vocal"]:
        parts.append("ambient")
        parts.append("white noise")
        parts.append("-vocal")
    elif "vocal-heavy" in row["vocal"]:
        parts.append("vocal")
        parts.append("energetic")

    # 3) Energy-based 표현
    e_min, e_max = row["energy_min"], row["energy_max"]
    if e_max >= 0.65:
        parts.append("energetic")
    elif e_min <= 0.3:
        parts.append("calm")

    # 4) BPM 표현
    if pd.notna(row["bpm_min"]) and pd.notna(row["bpm_max"]):
        parts.append(f"{int(row['bpm_min'])}-{int(row['bpm_max'])} bpm")

    # 5) Unique 정리 + 연결
    tokens = list(dict.fromkeys(parts))  # 순서 유지 중복 제거
    return " ".join(tokens)

df["search_query"] = df.apply(make_search_query, axis=1)
df["search_url"] = "https://open.spotify.com/search/" + df["search_query"].str.replace(" ", "%20")

df.to_csv("rule_mapping_v2.csv", index=False, encoding="utf-8-sig")
print(df[["goal", "search_query", "search_url"]].head())


     goal                               search_query  \
0  active  pop edm active calm energetic 105-115 bpm   
1  active  pop edm active calm energetic 108-118 bpm   
2  active  pop edm active calm energetic 110-120 bpm   
3  active  pop edm active calm energetic 120-130 bpm   
4  active  pop edm active calm energetic 105-115 bpm   

                                          search_url  
0  https://open.spotify.com/search/pop%20edm%20ac...  
1  https://open.spotify.com/search/pop%20edm%20ac...  
2  https://open.spotify.com/search/pop%20edm%20ac...  
3  https://open.spotify.com/search/pop%20edm%20ac...  
4  https://open.spotify.com/search/pop%20edm%20ac...  


**rule_mapping_v1과 달라진점**

playlist_seed 대신 → search_query 자동 생성

search_querysms Spotify 검색엔진이 충분히 이해할 수 있는 “자연어 기반 복합 검색어”

goal + genre + mood + bpm + energy + vocal 을 다 반영

결과는 실제 Spotify /search URL로 변환 가능

-> 하지만 링크제공이나 앱내에서 미리보기, 썸네일 제공은 불가능

**rule_mapping__v2.5.csv** 생성

스포티파이 감성에 맞춘 다른 버전( 더 검색이 잘되는 문자로 변경함)

맥락 단어를 자연어로 바꾸고(“for focus / to relax / workout music / meditation music / sleep sounds / study music / background music”), 보컬·에너지·BPM 힌트, 불필요한 결과 제거용 -live -remix -karaoke -cover까지 자동 포함

make_search_query_v2(): Spotify 감성(자연어형, -live/-remix 포함)으로 업그레이드

In [13]:
import pandas as pd
from urllib.parse import quote

# 1) CSV 읽기
df = pd.read_csv("rule_mapping_v1.csv", encoding="utf-8-sig")

# 2) goal → 자연어 맥락 매핑 (Spotify 검색 친화)
GOAL_MAP = {
    "focus": "for focus",
    "reading": "study music",
    "relax": "to relax",
    "active": "workout music",
    "meditate": "meditation music",
    "sleep": "sleep sounds",
    "neutral": "background music",
}

def _safe(s):
    return "" if pd.isna(s) else str(s).strip()

def _float(x):
    try: return float(x)
    except: return None

def make_search_query_v2(row):
    parts = []

    # --- 0) 장르(핵심 키워드 먼저)
    gp = _safe(row.get("genre_primary"))
    gs = _safe(row.get("genre_secondary"))
    if gp: parts.append(gp)
    if gs and gs.lower() != gp.lower(): parts.append(gs)

    # --- 1) 보컬 정책
    vocal = _safe(row.get("vocal")).lower()
    if "instrumental" in vocal:
        parts += ["instrumental", "-live", "-remix", "-karaoke", "-cover"]
    elif "no" in vocal:  # no-vocal
        parts += ["ambient", "white noise", "-vocal", "-live", "-remix", "-karaoke", "-cover"]
    elif "vocal-heavy" in vocal:
        parts += ["vocal"]

    # --- 2) 무드(감성) + 에너지 힌트
    mood = _safe(row.get("mood"))
    if mood: parts.append(mood)

    e_min, e_max = _float(row.get("energy_min")), _float(row.get("energy_max"))
    if e_min is not None and e_max is not None:
        if e_max >= 0.65:
            parts.append("energetic")
        elif e_min <= 0.30:
            parts.append("calm")
        # 중간값은 굳이 넣지 않음(노이즈 ↓)

    # --- 3) 맥락(Goal) 자연어
    goal_raw = _safe(row.get("goal")).lower()
    parts.append(GOAL_MAP.get(goal_raw, goal_raw or "background music"))

    # --- 4) BPM 힌트(자연어)
    bmin, bmax = _float(row.get("bpm_min")), _float(row.get("bpm_max"))
    if bmin is not None and bmax is not None:
        if int(bmin) == int(bmax):
            parts.append(f"{int(bmin)} bpm")
        else:
            parts.append(f"{int(bmin)}-{int(bmax)} bpm")

    # --- 5) 중복 제거 + 정리
    clean = []
    seen = set()
    for t in parts:
        t = t.strip()
        if not t: continue
        if t not in seen:
            clean.append(t)
            seen.add(t)

    return " ".join(clean)

# 3) 생성 & 링크
df["search_query"] = df.apply(make_search_query_v2, axis=1)
df["search_url"] = "https://open.spotify.com/search/" + df["search_query"].apply(lambda s: quote(s, safe=""))

# 4) 저장
out = "rule_mapping__v2.5.csv"
df.to_csv(out, index=False, encoding="utf-8-sig")
print("✅ saved:", out)
df[["goal","search_query","search_url"]].head(10)


✅ saved: rule_mapping__v2.5.csv


,goal,search_query,search_url
0,active,pop edm calm energetic workout music 105-115 bpm,https://open.spotify.com/search/pop%20edm%20ca...
1,active,pop edm calm energetic workout music 108-118 bpm,https://open.spotify.com/search/pop%20edm%20ca...
2,active,pop edm calm energetic workout music 110-120 bpm,https://open.spotify.com/search/pop%20edm%20ca...
3,active,pop edm calm energetic workout music 120-130 bpm,https://open.spotify.com/search/pop%20edm%20ca...
4,active,pop edm calm energetic workout music 105-115 bpm,https://open.spotify.com/search/pop%20edm%20ca...
5,focus,lo-fi instrumental hiphop instrumental -live -...,https://open.spotify.com/search/lo-fi%20instru...
6,focus,lo-fi instrumental hiphop instrumental -live -...,https://open.spotify.com/search/lo-fi%20instru...
7,focus,lo-fi instrumental hiphop instrumental -live -...,https://open.spotify.com/search/lo-fi%20instru...
8,focus,lo-fi instrumental hiphop calm energetic for f...,https://open.spotify.com/search/lo-fi%20instru...
9,focus,lo-fi instrumental hiphop instrumental -live -...,https://open.spotify.com/search/lo-fi%20instru...


## 사용자선호도 가중치를 더하는 3변수를 추가한 규칙매핑표v3


In [2]:
import pandas as pd

# 1️⃣ 기존 규칙매핑표 불러오기
# (CSV 파일이 같은 폴더에 있다고 가정)
df = pd.read_csv("rule_mapping__v2.5.csv")

# 2️⃣ 새 컬럼 추가 (기본값 설정)
df["pref_genre_bonus"] = 0.15       # 선호 장르 일치 시 가산점
df["pref_artist_bonus"] = 0.10      # 선호 아티스트 일치 시 가산점
df["seed_mix_strategy"] = "hybrid"  # Spotify 검색 시드 전략

# 3️⃣ 새로운 버전으로 저장
df.to_csv("rule_mapping__v3.csv", index=False, encoding="utf-8-sig")

print("✅ rule_mapping__v3.csv 파일이 생성되었습니다!")


✅ rule_mapping__v3.csv 파일이 생성되었습니다!
